In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix,accuracy_score, recall_score, precision_score, f1_score,classification_report 
from sklearn.metrics import roc_curve, auc

import model_team14 
from model_team14 import *

In [2]:
def make_auc_dict(model_name, clf, labels, y_test, y_pred, y_pred_prob):

    dict_auc={'model':[], 'class':[], 'auc':[]}

    # 1이면, 1/2이면 각각의 auc score

    if len(labels)==2:

        #class_probabilities = model.predict_proba(X_test)
#        preds = y_pred_prob[:, 1]

        fpr, tpr, threshold = roc_curve(y_test, y_pred)
        roc_auc = auc(fpr, tpr)

        dict_auc['model'].append(model_name)
        dict_auc['class'].append(0)
        dict_auc['auc'].append(roc_auc)

        dict_auc['model'].append(model_name)
        dict_auc['class'].append(1)
        dict_auc['auc'].append(roc_auc)

    else:
        #def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(17, 6)):
       ## y_score = ### model.predict_proba(X_test)

        #Binarize the output
        y_test_bin = label_binarize(y_test, classes=sorted(labels))
#        print(y_test_bin)
        n_classes = y_test_bin.shape[1]

        fpr = dict()
        tpr = dict()
        roc_auc = dict()
                                    
        for i in range(n_classes):
            if 'lstm' in model_name:
                fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i].detach().numpy())  #y_score[:, i]
            else:
                fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])  #y_score[:, i]
            dict_auc['model'].append(model_name)
            dict_auc['class'].append(i)
            dict_auc['auc'].append(auc(fpr[i], tpr[i]))

    return pd.DataFrame(dict_auc)

In [3]:
def make_otherreport():

    folders=[x for x in os.listdir("../result") if ('ft' in x)&('ls' not in x)] 

    for i, folder in enumerate(folders):
    #    print(folder)
        reports=[x for x in os.listdir("../result/{}".format(folder)) if 'creport' in x]
        models=[x for x in os.listdir("../result/{}".format(folder)) if 'clf' in x]
        ft_criteria=folder.split('_')[1]
        ft_threshold=folder.split('_')[2]

        path='../result/{}'.format(folder)

        for k, report in enumerate(reports):
            with open (path+'/{}'.format(report), 'rb') as f:
                df=pickle.load(f)
            if k==0:
                df_report=df
            else:    
                df_report=pd.concat([df_report, df])

        df_report['ft_criteria']=ft_criteria    
        df_report['ft_threshold']=ft_threshold
         

        for j, model in enumerate(models):
    #        print(model)
            model_name=model.split('.')[0]
            dtype=model.split('_')[4]
            y_type=model.split('_')[2]+'_'+model.split('_')[3]
            test_year=5

            with open (path+'/{}'.format(model), 'rb') as f:
                [clf, df_cvresult, y_pred, y_pred_prob, clf_report]=pickle.load(f)

            if dtype=='tr':
                X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
                y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
            else:
                X=pd.read_csv('../data/X_data.csv', index_col='date', parse_dates=True)
                y=pd.read_csv('../data/y_data.csv', index_col='date', parse_dates=True)

            labels=list(y[y_type].unique())    
            y_test=y[y_type][-(test_year*12):]

##        print(labels, y_type)

            df_tmp=make_auc_dict(model_name, clf, labels, y_test, y_pred, y_pred_prob)
    ##        print('auc df completed')
    ##        print(df_tmp.head())

            if j==0:
                df_auc=df_tmp
            else:
                df_auc=pd.concat([df_auc, df_tmp])

        df_report=df_report.merge(df_auc, left_on=['class','model'], right_on=['class','model'])

        if i==0:
            df_final_report=df_report
        else:
            df_final_report=pd.concat([df_final_report, df_report])
            
    return df_final_report            

In [4]:
df_report=make_otherreport()

df_report.head()
len(df_report)

160

In [5]:
df_report.head()

,class,precision,recall,f1_score,support,accuracy,model,ft_criteria,ft_threshold,auc
0,0.0,0.98,0.82,0.89,56.0,0.80,clf_knn_y_agg_tr_block_t5_spl3,None,0.2,0.872768
1,1.0,0.10,0.50,0.17,2.0,0.80,clf_knn_y_agg_tr_block_t5_spl3,None,0.2,0.818966
2,2.0,0.33,0.50,0.40,2.0,0.80,clf_knn_y_agg_tr_block_t5_spl3,None,0.2,0.961207
3,0.0,0.93,1.00,0.97,56.0,0.93,clf_knn_y_agg_ntr_block_t5_spl3,None,0.2,0.625000
4,1.0,0.00,0.00,0.00,2.0,0.93,clf_knn_y_agg_ntr_block_t5_spl3,None,0.2,0.491379


In [6]:
df_others_report=df_report.groupby(['ft_criteria','ft_threshold','model']).mean().reset_index().drop(columns=['class','f1_score','support'])

In [12]:
with open('../result/df_lstm_creport.pkl', 'rb') as f:
    df_lstm_report=pickle.load(f)

df_lstm_report.sort_values(['auc'], ascending=[False])

,model,precision,recall,accuracy,auc,ft_criteria,ft_threshold
1,clf_lstm_y_oecd_18_b5_ep300_h100,0.857143,0.857143,0.857143,0.886621,None,0.2
5,clf_lstm_y_oecd_24_b5_ep300_h100,0.861111,0.861111,0.861111,0.885802,None,0.2
6,clf_lstm_y_oecd_24_b5_ep500_h50,0.777778,0.777778,0.777778,0.858025,None,0.2
3,clf_lstm_y_agg_18_b5_ep500_h100,0.690476,0.690476,0.690476,0.851191,None,0.2
4,clf_lstm_y_agg_24_b5_ep300_h50,0.777778,0.777778,0.777778,0.848572,None,0.3
...,...,...,...,...,...,...,...
2,clf_lstm_y_oecd_18_b5_ep500_h50,0.523810,0.523810,0.523810,0.523243,None,0.3
6,clf_lstm_y_oecd_24_b5_ep500_h50,0.444444,0.444444,0.444444,0.513889,None,0.4
5,clf_lstm_y_oecd_24_b5_ep300_h100,0.444444,0.444444,0.444444,0.495370,None,0.3
4,clf_lstm_y_oecd_24_b5_ep300_h50,0.416667,0.416667,0.416667,0.482253,None,0.5


In [8]:
df_final_report=pd.concat([df_others_report, df_lstm_report])
len(df_final_report)==len(df_others_report)+len(df_lstm_report)

True

In [9]:
df_final_report.sort_values(['recall','precision','auc'], ascending=[False, False, False])[:10]

,ft_criteria,ft_threshold,model,precision,recall,accuracy,auc
10,None,0.2,clf_rf_y_agg_tr_block_t5_spl3,0.566667,0.940000,0.830000,0.955229
38,None,0.4,clf_knn_y_oecd_tr_block_t5_spl3,0.645000,0.910000,0.830000,0.910714
39,None,0.4,clf_knn_y_oecd_tr_tss_t5_spl3,0.645000,0.910000,0.830000,0.910714
54,None,0.5,clf_knn_y_oecd_tr_block_t5_spl3,0.635000,0.900000,0.820000,0.901786
55,None,0.5,clf_knn_y_oecd_tr_tss_t5_spl3,0.635000,0.900000,0.820000,0.901786
5,None,0.2,clf_lstm_y_oecd_24_b5_ep300_h100,0.861111,0.861111,0.861111,0.885802
1,None,0.2,clf_lstm_y_oecd_18_b5_ep300_h100,0.857143,0.857143,0.857143,0.886621
62,None,0.5,clf_rf_y_oecd_tr_block_t5_spl3,0.595000,0.850000,0.720000,0.848214
63,None,0.5,clf_rf_y_oecd_tr_tss_t5_spl3,0.595000,0.850000,0.720000,0.848214
4,None,0.2,clf_lstm_y_oecd_24_b5_ep300_h50,0.805556,0.805556,0.805556,0.846451


In [10]:
# df=df_final_report.groupby(['ft_criteria','ft_threshold','model']).mean() ##[['auc']].reset_index()
# df_selection=df.reset_index().sort_values(['recall','precision','auc'], ascending=[False, False, False])[:10]
# df_selection.head(10)

# df=df_final_report[df_final_report['class']>0].groupby(['ft_criteria','ft_threshold','model']).mean() ##[['auc']].reset_index()
# df_selection=df.reset_index().sort_values(['recall','precision','auc'], ascending=[False, False, False])[:10]
# df_selection.head(10)



# # df3=df_final_report.groupby(['ft_criteria','ft_threshold','model']).mean() ##[['auc']].reset_index()
# # df3.reset_index().sort_values(['recall','f1_score','precision','accuracy'], ascending=[False, False, False, False])[:50]

# # df1=df_final_report.groupby(['ft_criteria','ft_threshold','model']).max('class') ##[['auc']].reset_index()
# # df1.reset_index().sort_values(['auc','recall','precision','accuracy'], ascending=[False, False, False, False])[:30]


In [11]:
# df_selection['model_type']=df_selection['model'].apply(lambda x:x.split('_')[1])
# df_selection.head(10)